In [1]:
"""
分析non-LTR-RT数据的所有卷积层 (Conv1, Conv2, Conv3)
生成与LTR-RT数据一致风格的热图用于对比
不进行特定区域的选择性分析，仅展示整体特征分布
"""

import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

def tensor_to_numpy(tensor):
    """
    使用tolist()方法安全转换tensor到numpy
    这是最可靠的方法
    """
    # 如果已经是numpy数组，直接返回
    if isinstance(tensor, np.ndarray):
        return tensor
    # 如果是Python列表，转为numpy
    if isinstance(tensor, (list, tuple)):
        return np.array(tensor)
    # 处理PyTorch tensor - 使用tolist()方法
    if isinstance(tensor, torch.Tensor):
        # 先detach和cpu（如果需要）
        if tensor.requires_grad:
            tensor = tensor.detach()
        if tensor.is_cuda:
            tensor = tensor.cpu()
        # 使用tolist()转为Python列表，再转numpy
        return np.array(tensor.tolist())
    # 其他情况
    return np.array(tensor)

# ==================== 配置 ====================
# 各层的映射参数（与原脚本保持一致）
LAYER_CONFIG = {
    'Conv1': {
        'index': 1,
        'stride': 10,
        'channels': 32,
        'name': 'Conv1 (after MaxPool 10x)'
    },
    'Conv2': {
        'index': 4,
        'stride': 150,  # 10 * 15
        'channels': 64,
        'name': 'Conv2 (after MaxPool 150x)'
    },
    'Conv3': {
        'index': 7,
        'stride': 2250,  # 10 * 15 * 15
        'channels': 128,
        'name': 'Conv3 (after MaxPool 2250x)'
    }
}

# ==================== 加载数据 ====================
print("加载non-LTR-RT数据...")
mid_res = torch.load('non-LTR_RTtensor_data.pt')

# 检查数据结构
print(f"\nmid_res类型: {type(mid_res)}")
print(f"mid_res长度: {len(mid_res)}")
print("\n前几个元素的信息:")
for i in range(min(10, len(mid_res))):
    item = mid_res[i]
    if isinstance(item, torch.Tensor):
        print(f"  [{i}] Tensor: shape={item.shape}, device={item.device}")
    elif isinstance(item, (list, tuple)):
        print(f"  [{i}] List/Tuple: 长度={len(item)}")
        if len(item) > 0 and isinstance(item[0], torch.Tensor):
            print(f"       第一个元素: shape={item[0].shape}, device={item[0].device}")
    else:
        print(f"  [{i}] 其他类型: {type(item)}")

# ==================== 分析函数 ====================
def analyze_layer(layer_name, layer_config, mid_res):
    """分析单个卷积层"""
    print(f"\n{'='*60}")
    print(f"分析 {layer_name}")
    print(f"{'='*60}")
    
    try:
        # 提取数据 - 处理可能的嵌套结构
        layer_output = mid_res[layer_config['index']]
        print(f"原始数据类型: {type(layer_output)}")
        
        # 如果是列表或元组，取第一个元素
        if isinstance(layer_output, (list, tuple)):
            print(f"  是序列类型，长度={len(layer_output)}")
            tensor = layer_output[0]
        else:
            tensor = layer_output
        
        print(f"Tensor类型: {type(tensor)}")
        print(f"Tensor形状: {tensor.shape}")
        print(f"Tensor设备: {tensor.device}")
        print(f"需要梯度: {tensor.requires_grad}")
        
        # 转换为numpy
        print("正在转换为numpy...")
        data = tensor_to_numpy(tensor)
        print(f"✅ 转换成功！numpy形状: {data.shape}")
        
        # 关键：去除多余的维度 (1, 32, 1, 49981) -> (32, 49981)
        print(f"原始形状: {data.shape}")
        while data.ndim > 2:
            # 找到大小为1的维度并squeeze
            if 1 in data.shape:
                data = np.squeeze(data)
                print(f"  squeeze后: {data.shape}")
            else:
                # 如果没有大小为1的维度，可能需要reshape
                # 假设格式是 (batch, channels, height, width)
                # 取第一个batch
                if data.shape[0] == 1:
                    data = data[0]
                    print(f"  取第一个batch: {data.shape}")
                else:
                    break
        
        # 确保是2D (channels, positions)
        if data.ndim == 3:
            # 如果还是3D，可能是 (channels, 1, positions)
            data = data.squeeze()
            print(f"  最终squeeze: {data.shape}")
        
        print(f"✅ 最终形状: {data.shape}")
        
        stride = layer_config['stride']
        
        print(f"\n特征图形状: {data.shape}")
        print(f"分辨率: 每个特征位置 ≈ {stride} bp")
        
        # 归一化（使用与原脚本相同的方法）
        print("正在归一化...")
        scaler = preprocessing.RobustScaler()
        data_norm = scaler.fit_transform(data.T).T
        print("✅ 归一化完成")
        
        # 计算基本统计信息
        print(f"\n激活统计:")
        print(f"  均值: {data.mean():.4f}")
        print(f"  标准差: {data.std():.4f}")
        print(f"  最小值: {data.min():.4f}")
        print(f"  最大值: {data.max():.4f}")
        
        return {
            'data': data,
            'data_norm': data_norm,
        }
        
    except Exception as e:
        print(f"\n❌ 错误: {e}")
        print(f"错误类型: {type(e).__name__}")
        import traceback
        traceback.print_exc()
        return None

# ==================== 分析所有层 ====================
results = {}
for layer_name, config in LAYER_CONFIG.items():
    result = analyze_layer(layer_name, config, mid_res)
    if result is not None:
        results[layer_name] = result
    else:
        print(f"\n⚠️ {layer_name} 分析失败，跳过")

# 检查是否有成功的结果
if len(results) == 0:
    print("\n❌ 所有层都分析失败！")
    print("请检查数据格式和索引是否正确")
    exit(1)

print(f"\n✅ 成功分析了 {len(results)} 个层")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/proj/nobackup/hpc2nstor2024-028/zhychen/miniconda3/envs/py310/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/proj/nobackup/hpc2ns

加载non-LTR-RT数据...

mid_res类型: <class 'list'>
mid_res长度: 17

前几个元素的信息:
  [0] Tensor: shape=torch.Size([1, 1, 5, 50000]), device=cpu
  [1] Tensor: shape=torch.Size([1, 32, 1, 49981]), device=cpu
  [2] Tensor: shape=torch.Size([1, 32, 1, 49981]), device=cpu
  [3] Tensor: shape=torch.Size([1, 32, 1, 4998]), device=cpu
  [4] Tensor: shape=torch.Size([1, 64, 1, 4979]), device=cpu
  [5] Tensor: shape=torch.Size([1, 64, 1, 4979]), device=cpu
  [6] Tensor: shape=torch.Size([1, 64, 1, 331]), device=cpu
  [7] Tensor: shape=torch.Size([1, 128, 1, 297]), device=cpu
  [8] Tensor: shape=torch.Size([1, 128, 1, 297]), device=cpu
  [9] Tensor: shape=torch.Size([1, 128, 1, 19]), device=cpu

分析 Conv1
原始数据类型: <class 'torch.Tensor'>
Tensor类型: <class 'torch.Tensor'>
Tensor形状: torch.Size([1, 32, 1, 49981])
Tensor设备: cpu
需要梯度: False
正在转换为numpy...
✅ 转换成功！numpy形状: (1, 32, 1, 49981)
原始形状: (1, 32, 1, 49981)
  squeeze后: (32, 49981)
✅ 最终形状: (32, 49981)

特征图形状: (32, 49981)
分辨率: 每个特征位置 ≈ 10 bp
正在归一化...
✅ 归一化完成

激活统计:


In [4]:
# ==================== 生成无标亮的纯净热图 ====================
print(f"\n{'='*60}")
print("生成non-LTR-RT纯净热图...")
print(f"{'='*60}\n")

for layer_name, result in results.items():
    data_width = result['data'].shape[1]
    data_channels = result['data'].shape[0]
    
    # 统一图片尺寸 - 所有图都方正（与原脚本完全一致）
    fig_width = 12
    fig_height = 10
    
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))
    
    # 绘制热图（使用与原脚本完全相同的参数）
    im = ax.imshow(result['data_norm'], aspect='auto', cmap='RdYlBu_r',
                   interpolation='nearest', vmin=-2, vmax=2)
    
    # 计算层描述
    if layer_name == 'Conv1':
        layer_desc = 'Conv1'
    elif layer_name == 'Conv2':
        layer_desc = 'Conv2'
    else:
        layer_desc = 'Conv3'
    
    # 标题 - 标注为non-LTR-RT数据
    ax.set_title(f"{layer_desc} without LTR-RT Regions\n"
                 f"Shape: {data_channels} channels × {data_width} positions",
                 fontsize=30, fontweight='bold', pad=15)
    
    ax.set_xlabel('Feature Map Position', fontsize=25)
    ax.set_ylabel('Channel', fontsize=25)
    ax.tick_params(axis='both', labelsize=25)
    # 颜色条（与原脚本完全一致）
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Normalized Activation', rotation=270, labelpad=20, fontsize=25)
    
    # 网格 - 根据通道数调整（与原脚本完全一致）
    if data_channels <= 64:
        ytick_step = max(1, data_channels // 16)
    else:
        ytick_step = max(1, data_channels // 20)
    
    ax.set_yticks(np.arange(0, data_channels, ytick_step))
    ax.grid(True, alpha=0.2, axis='y', linestyle=':')
    
    plt.tight_layout()
    
    # 保存 - 文件名标注non-LTR
    filename = f'{layer_name}_non-LTR_heatmap_clean.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"✅ {layer_name} non-LTR-RT热图已保存: {filename}")
    print(f"   - 图片尺寸: {fig_width}×{fig_height} inches")
    print(f"   - 特征图: {data_channels} channels × {data_width} positions\n")
    
    plt.close()

print(f"{'='*60}")
print("✅ 完成！所有non-LTR-RT热图已生成")
print(f"{'='*60}\n")
print("生成的文件:")
for layer_name in results.keys():
    print(f"  - {layer_name}_non-LTR_heatmap_clean.png")


生成non-LTR-RT纯净热图...

✅ Conv1 non-LTR-RT热图已保存: Conv1_non-LTR_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 32 channels × 49981 positions

✅ Conv2 non-LTR-RT热图已保存: Conv2_non-LTR_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 64 channels × 4979 positions

✅ Conv3 non-LTR-RT热图已保存: Conv3_non-LTR_heatmap_clean.png
   - 图片尺寸: 12×10 inches
   - 特征图: 128 channels × 297 positions

✅ 完成！所有non-LTR-RT热图已生成

生成的文件:
  - Conv1_non-LTR_heatmap_clean.png
  - Conv2_non-LTR_heatmap_clean.png
  - Conv3_non-LTR_heatmap_clean.png
